In [1]:
from fredapi import Fred
import pandas as pd
from datetime import *
from pandas.tseries.offsets import MonthEnd

In [2]:
m2_prev = pd.read_excel(r'..\01-Raw_data\03-M2\m2_billions.usd_1867_1970_anually.xlsx')

In [3]:
m2_prev = m2_prev[['Year', 'M2']]
m2_prev = m2_prev.dropna()
m2_prev.rename(columns={"M2": "Value"}, inplace=True)
m2_prev['Year'] = m2_prev['Year'].astype(int)
m2_prev['Year'] = pd.to_datetime(m2_prev['Year'], format='%Y')
m2_prev = m2_prev.set_index('Year')

In [4]:
mask = (m2_prev.index < datetime.strptime('1959-01-01', '%Y-%m-%d'))
m2_prev = m2_prev.loc[mask]
m2_prev.index = m2_prev.index.map(lambda x: pd.datetime(x.year, 12, x.day))
m2_prev.index = pd.to_datetime(m2_prev.index, format='%Y') + MonthEnd(1)

<ipython-input-4-7427b4ec9383>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  m2_prev.index = m2_prev.index.map(lambda x: pd.datetime(x.year, 12, x.day))


In [5]:
m2_prev

,Value
Year,
1958-12-31,302.93
1957-12-31,287.06
1956-12-31,274.89
1955-12-31,264.81
1954-12-31,251.84
...,...
1871-12-31,2.45
1870-12-31,2.21
1869-12-31,2.05


In [15]:
fred = Fred(api_key='a7eca89fdf2905baea21d67b942c9ef7')
m2=fred.get_series('M2SL')
m2

1959-01-01      286.6
1959-02-01      287.7
1959-03-01      289.2
1959-04-01      290.1
1959-05-01      292.2
               ...   
2021-01-01    19395.4
2021-02-01    19666.7
2021-03-01    19912.8
2021-04-01    20118.8
2021-05-01    20370.1
Length: 749, dtype: float64

In [16]:
m2 = pd.DataFrame(data=m2, columns=['Value'])
m2 = m2.asfreq(freq='YS', method='ffill')
m2

,Value
1959-01-01,286.6
1960-01-01,298.2
1961-01-01,314.1
1962-01-01,337.5
1963-01-01,365.2
...,...
2017-01-01,13295.3
2018-01-01,13876.1
2019-01-01,14446.8
2020-01-01,15410.0


In [17]:
m2_def = m2.append(m2_prev).sort_index().reset_index().set_index('index')
m2_def

,Value
index,
1867-12-31,1.83
1868-12-31,1.85
1869-12-31,2.05
1870-12-31,2.21
1871-12-31,2.45
...,...
2017-01-01,13295.30
2018-01-01,13876.10
2019-01-01,14446.80


In [ ]:
m2_def.to_pickle(r'..\02-Data\06-M2\02_M2_USD_1867-12-31-2021-01-01_yearly_fred.pkl')

In [3]:
m2_def = pd.read_pickle(r'..\02-Data\06-M2\02_M2_USD_1867-12-31-2021-01-01_yearly_fred.pkl')

In [4]:
m2_pct = m2_def.pct_change(1).dropna()

In [5]:
m2_pct.to_pickle(r'..\02-Data\06-M2\01_M2_percentage_1868-12-31-2021-01-01_yearly_fred.pkl')

In [ ]:
m2_pct

In [ ]:
m2_prev_pct = m2_prev.pct_change(1)